In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
import os
import cv2

2024-07-04 20:31:21.479461: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 20:31:21.620104: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 20:31:21.795867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 20:31:21.967470: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 20:31:21.968860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 20:31:22.230794: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
def Image_Forgery_Predictor_Model():
    model = Sequential([
        Input(shape=(128, 128, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(2, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [3]:
def recomprimir_imagen_tf(imagen, calidad=50):
    imagen_jpeg = tf.image.encode_jpeg(imagen, quality=calidad)
    imagen_recomprimida = tf.image.decode_jpeg(imagen_jpeg)
    return imagen_recomprimida

def preprocess_image(image):
    image_compressed = recomprimir_imagen_tf(image, calidad=50)
    diff = tf.abs(tf.cast(image, tf.int32) - tf.cast(image_compressed, tf.int32))
    resized = tf.image.resize(diff, (128, 128))
    return resized



In [4]:
def train_model(images, labels, epochs=10):
    model = Image_Forgery_Predictor_Model()
    training_data = []
    for image in images:
        diff_image = preprocess_image(image)
        training_data.append(diff_image)
    training_data = np.array(training_data)
    labels = np.array(labels)
    
    # Dividir los datos en conjuntos de entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(training_data, labels, test_size=0.2, random_state=42)
    
    # Convertir las etiquetas a una representación binaria
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
    
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
    return model


In [6]:
# Path to your CASIA2 dataset
PATH = '../data/CASIA2'

# Directories for authentic and tampered images
authentic_dir = os.path.join(PATH, 'Au')
tampered_dir = os.path.join(PATH, 'Tp2')

authentic_number = 100
tampered_number = 100

def load_images_from_directory(directory_path, n, i=0):
    images = []
    for filename in os.listdir(directory_path)[i:n]:
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(directory_path, filename)
            imagen_original = tf.io.read_file(image_path)
            imagen_original = tf.image.decode_jpeg(imagen_original, channels=3)
            diff_image = preprocess_image(imagen_original)
            images.append(diff_image)
    return images

# Get file lists and labels
authentic_files = load_images_from_directory(authentic_dir, authentic_number)
tampered_files = load_images_from_directory(tampered_dir, tampered_number)
authentic_labels = [0] * len(authentic_files)
tampered_labels = [0] * len(tampered_files)

# Combine authentic and tampered data
all_files = authentic_files + tampered_files
all_labels = authentic_labels + tampered_labels

In [7]:
all_files = np.array(all_files)
all_labels = np.array(all_labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    all_files, all_labels, test_size=0.4, random_state=42
)

In [8]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [9]:
model = Image_Forgery_Predictor_Model()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.8817 - loss: 0.2666 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 999ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 946ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 957ms/step - accuracy: 1.0000 - loss: 0.00

In [10]:
# Suponiendo que tienes cargados tus datos en arreglos numpy:
# X_test es el arreglo de imágenes procesadas y y_test es el arreglo de etiquetas correspondientes

tests_number = 100

tampered_test = load_images_from_directory(authentic_dir, tests_number + tampered_number, i = tampered_number)
untampered_test = load_images_from_directory(tampered_dir, tests_number + authentic_number, i = authentic_number)
tampered_labels = [0] * len(tampered_test)
untampered_labels = [1] * len(untampered_test)

X_test = np.array(tampered_test + untampered_test)
y_test = np.array(tampered_labels + untampered_labels)

In [11]:
# Suponiendo que tienes tus datos en X_test y y_test como arreglos de numpy
# X_test contiene las imágenes preprocesadas
# y_test contiene las etiquetas correspondientes (0 o 1)

# Crear un Dataset a partir de los datos
dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Función para preprocesar las imágenes si es necesario
def preprocess_image(image, label):
    # Aquí puedes realizar cualquier preprocesamiento adicional necesario
    return image, label

# Aplicar el preprocesamiento al Dataset
dataset = dataset.map(preprocess_image)

# Definir el tamaño del lote y mezclar el Dataset (opcional)
batch_size = 32
dataset = dataset.batch(batch_size)

# Crear un iterador sobre el Dataset
iterator = iter(dataset)

# Lista para almacenar las predicciones y etiquetas reales
predictions = []

# Iterar sobre el Dataset para obtener predicciones
for batch in dataset:
    batch_images, batch_labels = batch
    
    # Obtener predicciones para el lote actual
    batch_predictions = model.predict(batch_images)
    
    # Iterar sobre las predicciones del lote y guardarlas junto con las etiquetas reales
    for i in range(len(batch_predictions)):
        input_image = batch_images[i]
        true_label = batch_labels[i]
        predicted_label = 1 if batch_predictions[i][0] > batch_predictions[i][1] else 0
        
        predictions.append((input_image, true_label, predicted_label))

# Evaluar el rendimiento comparando las predicciones con las etiquetas reales
correct_predictions = 0
total_images = len(predictions)

for input_image, true_label, predicted_label in predictions:
    if true_label == predicted_label:
        correct_predictions += 1

accuracy = correct_predictions / total_images
print(f"Accuracy: {accuracy:.2f}")

# Opcional: Imprimir algunas predicciones para verificar
for input_image, true_label, predicted_label in predictions[:100]:  # Imprimir las primeras 10 predicciones
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Accuracy: 0.50
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1
True Label: 0, Predicted Label: 1

2024-07-04 20:32:18.634994: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
